In [1]:
# pip install beautifulsoup4
# !pip install geopy
# !pip install gpxpy
# !pip install mplleaflet
# !pip install neo4j
# !pip install python-igraph

In [2]:
import gpxpy
import pandas as pd
import geopy
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import mplleaflet
import numpy as np
import glob
import datetime
from datetime import datetime
from datetime import timedelta
import matplotlib.colors as mcolors

In [3]:
gpx_arr=[]
segments=[]
dado = []
for name in glob.glob('gpx/*.gpx'):
    print(name)
    gpx_arr.append(gpxpy.parse(open(name)))
    
for gpx0 in gpx_arr:
    segments.append(gpx0.tracks[0].segments[0])
    
for seg in segments:
    for point_idx, point in enumerate(seg.points):
        dado.append([point.longitude, point.latitude,
                 point.elevation, point.time, seg.get_speed(point_idx)]) 

gpx/20190407.gpx
gpx/20191008.gpx
gpx/20191020.gpx
gpx/20190907.gpx
gpx/20191022.gpx
gpx/20191006.gpx
gpx/20190112.gpx
gpx/20190903.gpx
gpx/20190428.gpx
gpx/20190113.gpx
gpx/20191018.gpx
gpx/20190823.gpx
gpx/20190902.gpx
gpx/20191009.gpx
gpx/20191017.gpx
gpx/20190429.gpx
gpx/20190502.gpx
gpx/20191016.gpx
gpx/20191025.gpx
gpx/20190526.gpx
gpx/20191021.gpx
gpx/20191007.gpx
gpx/20190906.gpx
gpx/20190304.gpx
gpx/20190306.gpx
gpx/20190904.gpx
gpx/20190905.gpx
gpx/20190430.gpx
gpx/20190501.gpx
gpx/20190408.gpx
gpx/20190111.gpx
gpx/20191019.gpx
gpx/20191004.gpx
gpx/20191005.gpx


In [4]:
columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
df = pd.DataFrame(dado, columns=columns)
df.sort_values(by=['Time'], inplace=True)
df['dia_da_semana'] = df.Time.dt.day_name(locale='pt_BR.utf8')

In [5]:
def monta(df):
    lo_origem = []
    la_origem = []
    lo_destino = []
    la_destino = []
    velocidade_media = []
    distancia = []
    altitude_origem = []
    altitude_destino = []
    hora_origem = []
    hora_destino = []
    duracao = []
    data = []
    dia_da_semana = []
    for idx in range(1, len(df)):
        lo_origem.append(df.iloc[idx-1].Longitude)
        la_origem.append(df.iloc[idx-1].Latitude)
        lo_destino.append(df.iloc[idx].Longitude)
        la_destino.append(df.iloc[idx].Latitude)
        distancia.append(geodesic([df.iloc[idx-1].Longitude,df.iloc[idx-1].Latitude],[df.iloc[idx].Longitude, df.iloc[idx].Latitude]).km)
        velocidade_media.append(np.mean([df.iloc[idx-1].Speed,df.iloc[idx].Speed]))
        altitude_origem.append(df.iloc[idx-1].Altitude)
        altitude_destino.append(df.iloc[idx].Altitude)
        data.append(df.iloc[idx]['data'])
        dia_da_semana.append(df.iloc[idx].dia_da_semana)
        hora_origem.append(df.iloc[idx-1].hora)
        hora_destino.append(df.iloc[idx].hora)
        origem = df.iloc[idx-1].hora
        destino = df.iloc[idx].hora
        td = timedelta(hours=destino.hour, minutes=destino.minute, seconds=destino.second)
        to = timedelta(hours=origem.hour, minutes=origem.minute, seconds=origem.second)
        duracao.append(td-to)
    df_trk = pd.DataFrame()
    df_trk['lo_origem'] = lo_origem
    df_trk['la_origem'] = la_origem

    df_trk['lo_destino'] = lo_destino
    df_trk['la_destino'] = la_destino

    df_trk['velocidade_media'] = velocidade_media
    df_trk['distancia'] = distancia

    df_trk['altitude_origem'] = altitude_origem
    df_trk['altitude_destino'] = altitude_destino
    df_trk['data'] = data
    df_trk['dia_da_semana'] = dia_da_semana
    df_trk['hora_origem'] = hora_origem
    df_trk['hora_destino'] = hora_destino
    df_trk['duracao'] = duracao
    
#     df_trk = df_trk.fillna(0)
    return df_trk

In [6]:
df['data']=df.Time.dt.date
df['hora']=df.Time.dt.time
df.drop(columns=['Time'],inplace=True)
dias = list(df.data.unique())
trilhas = []
for dia in dias:
    if(len(df.loc[df.data == dia])>=10):
        trilhas.append(monta(df.loc[df.data == dia]))    

In [7]:
df_trk = pd.DataFrame()
for d in trilhas:
    df_trk = df_trk.append(d, ignore_index = True) 

In [8]:
df_trk.groupby(by=['data'])['data'].count()

data
2019-01-11    345
2019-01-12    126
2019-01-13     52
2019-03-06     46
2019-04-07     78
2019-04-08     89
2019-04-28     53
2019-04-29     82
2019-04-30     76
2019-05-01     90
2019-05-02     78
2019-05-26     69
2019-09-03     59
2019-09-04     85
2019-09-05     91
2019-09-06     78
2019-09-07     77
2019-10-04     31
2019-10-05    120
2019-10-06    100
2019-10-07    106
2019-10-08     70
2019-10-09     73
2019-10-16     66
2019-10-17     85
2019-10-18     59
2019-10-19    142
2019-10-20    137
2019-10-21     79
2019-10-22     52
2019-10-25     69
Name: data, dtype: int64

In [9]:
[(x.day,x.month, x.year) for x in df.data.to_numpy() if(x.month == 1)]

[(11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1, 2019),
 (11, 1,

In [10]:
df.head(1)

,Longitude,Latitude,Altitude,Speed,dia_da_semana,data,hora
2138,-38.463048,-12.998856,NaN,0.034324,Sexta,2019-01-11,15:55:52.134000


In [11]:
df_jan[(df_jan['Longitude']==-38.463) & (df_jan['Latitude']==-12.9989)].groupby(by='dia_da_semana')['dia_da_semana'].count()

NameError: name 'df_jan' is not defined

In [ ]:
df_jan['Latitude'].value_counts().head()

In [ ]:
df_jan['Longitude'].value_counts().head()